In [1]:
from congress import Congress
api_key = 'fxzKX1XVwgCBfElNZlya7fwBVdEFNBlGEadiAFxD'
congress_api = Congress(api_key)
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer
pd.set_option('display.max_columns', None)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
import csv
import json
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
%matplotlib inline
import seaborn as sns
sns.set(font_scale=1.5)
import numpy as np
import importlib
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import CleanData, SaveState
importlib.reload(CleanData)
importlib.reload(SaveState)
from CleanData import clean_comment, get_bag_of_words

[nltk_data] Downloading package stopwords to /home/zchao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Get stopword list
stop_words = stopwords.words('english')
number_strs = ['one', 'two', 'three', 'four', 'five', 'six', 'seven',\
'eight', 'nine', 'ten']
stop_words.extend(number_strs)

In [3]:
# Load tweet from members
filename = '../data/115_Senators_during_term.csv'
tweets_df = pd.read_csv(filename)
tweets_df = tweets_df.head(200000)

In [4]:
tweets_df = tweets_df.loc[tweets_df['full_text'].apply(type) == str]

In [5]:
tweets_df['user_name'].unique()

array(['Doug Jones', 'Cory Booker', 'Lindsey Graham', 'Kamala Harris',
       'Hatch', 'Luther Strange', 'Bernie Sanders',
       'Shelley Moore Capito', 'Maggie Hassan', 'Thom Tillis',
       'Jerry Moran', 'Tom Carper', 'Martin Heinrich', 'Pat Toomey',
       'Bill Nelson', 'Mike Crapo', 'Mike Rounds', 'Todd Young',
       'John Barrasso', 'Murphy', 'Steve Daines', 'John Thune',
       'Susan Collins', 'Patty Murray', 'John McCain', 'Dianne Feinstein',
       'Jack Reed', 'ChuckGrassley', 'Cortez Masto', 'McCaskill',
       'Lamar Alexander', 'Pat Roberts', 'Ben Cardin', 'Ron Wyden',
       'Dean Heller', 'Dick Durbin', 'Jeanne Shaheen',
       'Sheldon Whitehouse', 'Tammy Duckworth', 'Tom Cotton',
       'Bob Corker', 'Sherrod Brown', 'Tom Udall', 'Gary Peters',
       'Ted Cruz', 'Johnny Isakson', 'James Lankford', 'McConnell',
       'Mike Enzi', 'Jeff Flake', 'Kirsten Gillibrand', 'Bob Casey',
       'Rubio', 'Angus King', 'Inhofe', 'Chris Van Hollen', 'Rob Portman',
       'Thad

In [6]:
dem_train_users = ['Chuck Schumer', 'Elizabeth Warren', 'Bernie Sanders','Michael Bennet', \
                   'Cory Booker', 'Amy Klobuchar']
rep_train_users = ['McConnell', 'Rubio', 'John McCain', 'Ted Cruz']

In [7]:
test_users = tweets_df['user_name'].unique().tolist()
for user in (dem_train_users + rep_train_users):
    test_users.remove(user)

In [8]:
print('No. of democratic training tweets is',
tweets_df.loc[tweets_df['user_name'].isin(dem_train_users)].shape[0])
print('No. of republican training tweets is',
tweets_df.loc[tweets_df['user_name'].isin(rep_train_users)].shape[0])

No. of democratic training tweets is 15772
No. of republican training tweets is 10386


In [9]:
print('No. of testing tweets is',
tweets_df.loc[tweets_df['user_name'].isin(test_users)].shape[0])

No. of testing tweets is 173514


In [10]:
print(tweets_df.columns)

Index(['tweet_id', 'full_text', 'created_at', 'user_id', 'screen_name',
       'user_name', 'hashtags'],
      dtype='object')


In [11]:
tweets_df['clean_tweet'] = tweets_df['full_text'].apply(clean_comment)
bag_of_words, vectorizer = get_bag_of_words(tweets_df['clean_tweet'],ngram_range=(1,3), min_df=0.001)
print(bag_of_words.shape) 

(199672, 2666)


In [12]:
tweets_df.head(5)

,tweet_id,full_text,created_at,user_id,screen_name,user_name,hashtags,clean_tweet
0,940601256560783400,rt ashparso gdouglasjones i voted dougjones dougjonesforsenate,Tue Dec 12 15:16:29 +0000 2017,239548513,DougJones,Doug Jones,[],rt ashparso gdouglasjones i voted dougjones dougjonesforsenate
1,940625117176041500,today let’s get everyone to vote thank you joebiden,Tue Dec 12 16:51:18 +0000 2017,239548513,DougJones,Doug Jones,[],today let’s get everyone to vote thank you joebiden
2,939908592802648000,let us go to the polls to do our own work and remember that our life depends on it because it does congresswoman sewell,Sun Dec 10 17:24:05 +0000 2017,239548513,DougJones,Doug Jones,[],let us go to the polls to do our own work and remember that our life depends on it because it does congresswoman sewell
3,940583699728527400,today rightsideofhistory go vote,Tue Dec 12 14:06:43 +0000 2017,239548513,DougJones,Doug Jones,['RightSideOfHistory'],today rightsideofhistory go vote
4,940606216786665500,rt aldotcom just in case you were wondering alpolitics alsen,Tue Dec 12 15:36:12 +0000 2017,239548513,DougJones,Doug Jones,['alpolitics'; 'alsen'],rt aldotcom just in case you were wondering alpolitics alsen


In [14]:
# Train binary multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn import metrics
def get_binary_NB_model(bag_of_words, df):
    # Training data:
    class1_words = bag_of_words[df['user_name'].isin(dem_train_users),:]
    class2_words = bag_of_words[df['user_name'].isin(rep_train_users),:]
    train_tweets = np.concatenate((class1_words,class2_words))
    labels = np.concatenate((np.zeros(class1_words.shape[0]),np.ones(class2_words.shape[0])))
    nb = ComplementNB()
    nb.fit(train_tweets, labels)
    # # Performance on training data
    predictions = nb.predict(train_tweets)
    print('Training Accuracy: ' + str(sum(labels==predictions)/len(labels)))
    # Compute the error.
    tn, fp, fn, tp = metrics.confusion_matrix(labels,predictions).ravel()
    print(tn, fp, fn, tp)
    return nb

In [15]:
# Train a naive bayes model on tweets from specified training users from each party.
nb_model = get_binary_NB_model(bag_of_words, tweets_df)

MemoryError: Unable to allocate array with shape (26158, 2666) and data type float64